# 01 Attribute Network and Make Scenario Pickle

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.INFO)

## Remote I/O

In [4]:
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_12')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_12')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-09-09 13:35:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 13:35:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 13:35:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 13:35:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Data Reads

In [6]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)

2021-09-09 13:35:19, INFO: Reading RoadwayNetwork
2021-09-09 13:35:19, INFO: Reading RoadwayNetwork
2021-09-09 13:37:45, INFO: Read 1634790 links from D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\link.json
2021-09-09 13:37:45, INFO: Read 1634790 links from D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\link.json
2021-09-09 13:37:45, INFO: Read 644480 nodes from D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\node.geojson
2021-09-09 13:37:45, INFO: Read 644480 nodes from D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\node.geojson
2021-09-09 13:37:45, INFO: Read 869986 shapes from D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\shape.geojson
2021-09-09 13:37:45, INFO: Read 869986 shapes from D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\shape.geojson


In [7]:
roadway_net.links_df.model_link_id.nunique()

1634790

*Please note*: a third party library is used to read in the GTFS feed and it will generate warnings such as, `INFO: Removing calendar.txt from transit network config because file not found`. These warnings can be ignored, as the standard Network Wrangler transit network does not need these files. 

In [8]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2021-09-09 13:55:46, INFO: Read in transit feed from: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12
2021-09-09 13:55:46, INFO: Read in transit feed from: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12
2021-09-09 13:55:46, INFO: Removing calendar.txt from transit network config because file not found
2021-09-09 13:55:46, INFO: Removing calendar.txt from transit network config because file not found
2021-09-09 13:55:46, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-09-09 13:55:46, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-09-09 13:55:49, INFO: Removing transfers.txt from transit network config because file not found
2021-09-09 13:55:49, INFO: Removing transfers.txt from transit network config because file not found
2021-09-09 13:55:49, INFO: Removing feed_info.txt from transit network config because file not found
2021-09-09 13:55:49, INFO: 

## Attribute the Network

In [9]:
r_net = mtc.determine_number_of_lanes(
    roadway_network = roadway_net,
    parameters = parameters,
    network_variable = 'lanes'
)

r_net.links_df.lanes.value_counts()

2021-09-09 13:56:00, INFO: Determining number of lanes
2021-09-09 13:56:00, INFO: Determining number of lanes
2021-09-09 13:56:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 13:56:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 13:56:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 13:56:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:00:59, INFO: Finished determining number of lanes using variable: lanes
2021-09-09 14:00:59, INFO: Finished determining number of lanes using variable: lanes


1    1486418
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: lanes, dtype: int64

In [10]:
r_net = mtc.calculate_facility_type(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'ft')

r_net.links_df.ft.value_counts()

2021-09-09 14:01:00, INFO: Calculating Facility Type
2021-09-09 14:01:00, INFO: Calculating Facility Type
2021-09-09 14:01:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:01:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:01:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:01:00, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:37, INFO: Finished calculating roadway class variable: ft
2021-09-09 14:03:37, INFO: Finished calculating roadway class variable: ft


99    890174
7     538666
6      96883
5      58344
4      38332
3       6365
2       3252
1       2774
Name: ft, dtype: int64

In [11]:
r_net = mtc.calculate_useclass(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'useclass')

r_net.links_df.useclass.value_counts()

2021-09-09 14:03:38, INFO: Determining useclass
2021-09-09 14:03:38, INFO: Determining useclass
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Calculating and adding roadway network variable: useclass
2021-09-09 14:03:38, INFO: Calculating and adding roadway network variable: useclass
2021-09-09 14:03:38, INFO: Finished determining variable: useclass
2021-09-09 14:03:38, INFO: Finished determining variable: useclass


0    1634790
Name: useclass, dtype: int64

In [12]:
r_net = mtc.calculate_assignable(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'assignable')

r_net.links_df.assignable.value_counts()

2021-09-09 14:03:38, INFO: Determining assignable
2021-09-09 14:03:38, INFO: Determining assignable
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:38, INFO: Calculating and adding roadway network variable: assignable
2021-09-09 14:03:38, INFO: Calculating and adding roadway network variable: assignable
2021-09-09 14:03:47, INFO: Finished determining assignable using variable: assignable
2021-09-09 14:03:47, INFO: Finished determining assignable using variable: assignable


0.0    1245221
1.0     388313
Name: assignable, dtype: int64

In [13]:
r_net = mtc.calculate_transit(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'transit')

r_net.links_df.transit.value_counts()

2021-09-09 14:03:47, INFO: Determining transit
2021-09-09 14:03:47, INFO: Determining transit
2021-09-09 14:03:47, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:47, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:47, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:47, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:47, INFO: Calculating and adding roadway network variable: transit
2021-09-09 14:03:47, INFO: Calculating and adding roadway network variable: transit
2021-09-09 14:03:47, INFO: Finished determining transit-only variable: transit
2021-09-09 14:03:47, INFO: Finished determining transit-only variable: transit


0    1634040
1        750
Name: transit, dtype: int64

In [14]:
transit_net.feed.stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21881 entries, 0 to 21880
Data columns (total 20 columns):
stop_name              21881 non-null object
stop_lat               21881 non-null float64
stop_lon               21881 non-null float64
zone_id                8232 non-null object
agency_raw_name        21881 non-null object
stop_code              15289 non-null object
location_type          5475 non-null float64
parent_station         249 non-null object
stop_desc              409 non-null object
stop_url               261 non-null object
stop_timezone          126 non-null object
wheelchair_boarding    377 non-null float64
platform_code          52 non-null object
position               0 non-null object
direction              0 non-null object
* used by routes       124 non-null object
stop_id                21881 non-null object
osm_node_id            21212 non-null object
shst_node_id           21212 non-null object
model_node_id          21881 non-null object
dtypes: floa

In [15]:
r_net = mtc.calculate_farezone(
    roadway_network = r_net,
    transit_network = transit_net,
    parameters = parameters,
    network_variable = 'farezone',
)

r_net.nodes_df.farezone.value_counts()

2021-09-09 14:03:48, INFO: Determining farezone
2021-09-09 14:03:48, INFO: Determining farezone
2021-09-09 14:03:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:48, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 14:03:48, INFO: Calculating and adding roadway network variable: farezone
2021-09-09 14:03:48, INFO: Calculating and adding roadway network variable: farezone
2021-09-09 14:03:49, INFO: Finished determining variable: farezone
2021-09-09 14:03:49, INFO: Finished determining variable: farezone


110.0    3454
89.0     1527
82.0      226
72.0      207
103.0     179
         ... 
63.0        1
62.0        1
61.0        1
1.0         1
128.0       1
Name: farezone, Length: 203, dtype: int64

## Create a Scenario

In [16]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2021-09-09 14:03:49, INFO: Creating Scenario
2021-09-09 14:03:49, INFO: Creating Scenario


## Write to Disk

In [17]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))

In [18]:
pd.crosstab(r_net.links_df.ft, r_net.links_df.roadway)

roadway,cycleway,footway,motorway,motorway_link,primary,primary_link,residential,secondary,secondary_link,service,tertiary,tertiary_link,trunk,trunk_link
ft,,,,,,,,,,,,,,
1,0,0,2774,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,3252,0
3,0,0,0,4983,0,0,0,0,0,0,0,0,0,1382
4,0,0,0,0,15505,0,0,17933,0,0,4894,0,0,0
5,0,0,0,0,8432,0,0,24996,0,0,24034,0,882,0
6,0,0,0,0,2555,1506,0,25600,1566,0,65019,611,26,0
7,0,0,0,0,0,0,538666,0,0,0,0,0,0,0
99,98614,247489,0,0,0,0,0,0,0,541983,0,0,0,0


In [19]:
transit_net.feed.stops.columns

Index(['stop_name', 'stop_lat', 'stop_lon', 'zone_id', 'agency_raw_name',
       'stop_code', 'location_type', 'parent_station', 'stop_desc', 'stop_url',
       'stop_timezone', 'wheelchair_boarding', 'platform_code', 'position',
       'direction', '* used by routes', 'stop_id', 'osm_node_id',
       'shst_node_id', 'model_node_id'],
      dtype='object')

In [20]:
roadway_net.links_df[roadway_net.links_df.county == "Marin"].model_link_id.max()

8063197

In [21]:
roadway_net.nodes_df[roadway_net.nodes_df.model_node_id == 5030000]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
